In [1]:
from Connection import Neo4j
import pandas as pd

In [2]:
neo4j_conn = Neo4j()
driver = neo4j_conn.get_driver()

### Aggregators analysis
This function examinse deposists or withdrawals made during a lookback period of n days where multiple in and out transfer amounts aggregate above a desired threshold.

In [3]:
def get_aggregates_over_threshold(curDate, lookback, threshold, sort='totAmount', order='DESC', limit=10, skip=0):
    """
    This function examinse deposists or withdrawals made during a look-back period of n days 
    where multiple in and out transfer amounts aggregate above a desired threshold.

    Parameters:
        curDate : str (yyyy-mm-dd)
            Reference date for the analysis
        lookback : int
            Number of lookback days.
        threshold : int
            Desired threshold for deposists and withdrawals
        sort : str, default 'totAmount'
            Data field used to sort the result
        order : str, default 'DESC'
            Define the sprting order of the results. 'DESC' or 'ASC'
        limit : int, default 10
            Maximum number of records top records returned
        skip : int, defalut 0
            Number of records to be trimmed from the top
    
    Returns: list of lists
        Each item in the list contains [custName, accNum, transaction date, transaction type, total amount]

    """

    def read_neo4j(tx, curDate, lookback, threshold, sort, order, limit, skip):
        # Define the cypher statement
        cypher_query = """
            WITH date('{0}') as curDate, {1} as curThreshold
            WITH curThreshold, curDate, curDate-duration({{days:{2}}}) as curDate30
            MATCH (trf:BankTransfer)
            CALL {{
                WITH trf, curDate, curDate30, curThreshold
                MATCH (trf)<-[:SEND]-(acc:BankAccount)
                WHERE trf.transferDate <= curDate
                AND trf.transferDate >= curDate30
                WITH acc.accNum as curAccNum, trf.transferDate as transactionDate, sum(trf.amount) as totAmount, curThreshold
                WHERE totAmount > curThreshold
                RETURN curAccNum, transactionDate, totAmount, 'Debit' as transactionType
                UNION ALL
                WITH trf, curDate, curDate30, curThreshold
                MATCH (trf)-[:SEND]->(acc:BankAccount)
                WHERE trf.transferDate <= curDate
                AND trf.transferDate >= curDate30
                WITH acc.accNum as curAccNum, trf.transferDate as transactionDate, sum(trf.amount) as totAmount, curThreshold
                WHERE totAmount > curThreshold
                RETURN curAccNum, transactionDate, totAmount, 'Credit' as transactionType 
            }}
            WITH curAccNum, transactionDate, totAmount, transactionType
            MATCH (ah:AccHolder)-[:HAS_BANKACCOUNT]->(ba:BankAccount {{accNum: curAccNum}})
            RETURN ah.name AS custName, curAccNum as accNum, transactionDate, totAmount, transactionType
            ORDER by {3} {4}
            SKIP $skip
            LIMIT $limit
        """.format(curDate, threshold, lookback, sort, order)
        
        # Run the statement within the transaction
        result = tx.run(cypher_query, limit=limit, skip=skip)

        # Extract records from the Result
        return result.values('custName','accNum', 'transactionDate', 'transactionType', 'totAmount')

    with driver.session() as session:
        return session.execute_read(read_neo4j, curDate, lookback, threshold, sort, order, limit, skip)

The below analysis looks back at a period of 30 days from '2021-06-30' <br>
and identify multiple in and out transfer amounts aggregate above 150k.

Note: *`curDate` should usually be today's date in a real-world scenario. <br>
For this analysis, we need to specify an old date 
as the dataset only contains transactions in 2021.*

In [4]:
aggregators = get_aggregates_over_threshold(curDate='2021-06-30'
                                            , lookback=30
                                            , threshold=150000
                                            )   

In [5]:
# Display the results
aggregatorsDF = pd.DataFrame(aggregators, columns = ['custName','accNum', 'transactionDate', 'transactionType', 'totAmount'])
display(aggregatorsDF)

,custName,accNum,transactionDate,transactionType,totAmount
0,Lillian Drummond,745-50-1433,2021-06-01,Debit,194113.9566
1,Adina Dallas,671-30-0017,2021-06-01,Credit,194113.9566
2,Jack Larsen,258-61-0744,2021-06-30,Debit,193845.4778
3,Hank Griffiths,514-65-8714,2021-06-30,Credit,193845.4778
4,Karla Underhill,332-18-5087,2021-06-01,Debit,190064.9513
5,Ramon Umney,726-55-0272,2021-06-01,Credit,190064.9513
6,Makenzie Campbell,406-42-0705,2021-06-26,Credit,188441.5867
7,Carl Butler,034-82-4477,2021-06-26,Debit,188441.5867
8,Noah Ainsworth,883-66-3870,2021-06-25,Debit,187861.4329
9,Shannon Rixon,855-21-8447,2021-06-25,Credit,187861.4329


### Recency, Frequency, and Monetary (RFM) Analysis
This RFM analysis identifies the loyalty of the customers towards various merchants.<br>
These insights can be used to run personalized marketing campaigns with higher conversion rates.

In [6]:
def get_rfm_details(curDate, lookback, sort='monetory', order='DESC', limit=10, skip=0):
    """
    This function returns the RFM details of customers against different merchants based on their purchases.
    The analysis looks back at n days to calculate the values.

    Parameters:
        curDate : str (yyyy-mm-dd)
            Reference data for the analysis
        lookback : int
            Number of look-back days.
        sort : str, default 'monetory'
            Data field used to sort the result
        order : str, default 'DESC'
            Define the sprting order of the results. 'DESC' or 'ASC'
        limit : int, default 10
            Maximum number of records top records returned
        skip : int, defalut 0
            Number of records to be trimmed from the top
    
    Returns: list of lists

    """
    def read_neo4j(tx, curDate, lookback, sort, order, limit, skip):
        # Define the cypher statement
        cypher_query = """
            WITH date('{0}') as curDate
            WITH curDate, curDate-duration({{days:{1}}}) as curDate30
            MATCH(ah:AccHolder)-[:HAS_CREDITCARD]->(:CreditCard)<-[:WITH_CARD]-(p:Purchase)-[:FROM]->(m:Merchant)
            WHERE p.purchaseDate <= curDate AND p.purchaseDate >= curDate30
            WITh ah.name as custName,
                m.name as merchant,
                SUM(p.amount) as monetory,
                COUNT(DISTINCT p) as frequency,
                MIN(
                    duration.inDays(
                        p.purchaseDate, curDate
                    ).days
                ) AS recency
            RETURN custName, merchant, recency, frequency, monetory
            ORDER BY {2} {3}
            SKIP $skip
            LIMIT $limit
        """.format(curDate, lookback, sort, order)
        # Run the statement within the transaction
        result = tx.run(cypher_query, limit=limit, skip=skip)

        # Extract records from the Result
        return result.values('custName', 'merchant', 'recency', 'frequency', 'monetory')
    
    with driver.session() as session:
        return session.execute_read(read_neo4j, curDate, lookback, sort, order, limit, skip)

The analysis below looks back at last 30 days of purchase records from 2021-06-30
to obtain the recency, frequency and monetary values for customers against different merchants. 

In [7]:
rfm_details = get_rfm_details(curDate='2021-06-30', lookback=30)

In [8]:
# Create a Data Frame and display results
rfm_df = pd.DataFrame(rfm_details, columns = ['Cust Name','Merchant', 'Recency', 'Frequency', 'Monetory'])
display(rfm_df)

,Cust Name,Merchant,Recency,Frequency,Monetory
0,Quinn Powell,Comodo,17,3,48713.518080
1,Logan Vaughan,Carrefour,15,3,43072.916320
2,Rebecca Corbett,Vodafone,3,4,41816.635854
3,Matt Carter,It Smart Group,13,3,40166.636561
4,Percy Warner,Areon Impex,10,2,38546.050480
5,Carl Butler,Telekom,0,2,36678.670410
6,Hayden Garcia,Comodo,2,3,36485.574544
7,Ivette Allen,Demaco,0,3,36005.761640
8,Johnathan Overson,21st Century Fox,7,2,35952.072420
9,Stella Kelly,Demaco,8,2,35501.899690


### Layering Scheme Analysis
This analysis looks for passthrough payments (A)->(B)->(C)

In [9]:
def examine_layering(curDate, lookback, transferThreshold, sort='amtTransfer', order='DESC', limit=10, skip=0):
    """
    This function looks for passthrough payments through intermediaries (A)->(B)->(C).
    This analysis examines transfers greater than the desired threshold between
    A and B and within 75% between B and C.

    Parameters:
        curDate : str (yyyy-mm-dd)
            Reference data for the analysis
        lookback : int
            number of lookback days.
        transferThreshold : int
            The bank transfers above this transfer threshold are examined.
        sort : str, default 'amtTransfer'
            Data field used to sort the result.
        order : str, default 'DESC'
            Define the sprting order of the results. 'DESC' or 'ASC'
        limit : int, default 10
            Maximum number of records top records returned.
        skip : int, defalut 0
            Number of records to be trimmed from the top.

    Returns: list of lists
    """

    def read_neo4j(tx, curDate, lookback, transferThreshold, limit, skip):
        # Define the cypher statement
        cypher_query= """
        WITH date('{0}') as curDate
        WITH curDate, curDate-duration({{days:{1}}}) as curDate30
        MATCH p=(ah1:AccHolder)-[:HAS_BANKACCOUNT]->(ba1:BankAccount)-[:SEND]->(trf1:BankTransfer)-[:SEND]->(ba2:BankAccount)-[:SEND]->(trf2:BankTransfer)-[:SEND]->(ba3:BankAccount)<-[:HAS_BANKACCOUNT]-(ah2:AccHolder)
        WHERE trf1.transferDate <= curDate
            AND trf1.transferDate >= curDate30
            AND trf2.transferDate >= trf1.transferDate
            AND duration.inDays(trf1.transferDate, trf2.transferDate).days <=7
            AND trf2.amount > trf1.amount * 0.75
            AND trf2.amount < trf1.amount
            AND trf1.amount > {2}
        WITH ah1.cif as senderId, ah1.name as senderName
            , sum(trf1.amount) as amtTransfer, count(trf1) as numTransfers
            , sum(trf2.amount) as amtRcv, count(trf2) as numRcv
            , ah2.cif as rcvId, ah2.name as rcvName
        RETURN senderId, senderName, numTransfers, amtTransfer, rcvId, rcvName, amtRcv, numRcv
        ORDER by {3} {4}
        SKIP $skip
        LIMIT $limit        
        """.format(curDate, lookback, transferThreshold, sort, order)
        
        # Run the statement within the transaction
        result = tx.run(cypher_query, limit=limit, skip=skip)

        # Extract records from the Result
        return result.values('senderId', 'senderName', 'numTransfers', 'amtTransfer', 'rcvId', 'rcvName', 'numRcv', 'amtRcv')
    
    with driver.session() as session:
        return session.execute_read(read_neo4j, curDate, lookback, transferThreshold, limit, skip)
        

The analysis below examines transfer above 1k over the last 30 days from 2021-06-30 to find passthrough payments.

In [10]:
layering = examine_layering(curDate='2021-06-30', lookback=30, transferThreshold=1000)

In [11]:
layering_df = pd.DataFrame(layering, columns = ['Sender Id','Sender Name', 'Num Transfers', 
                                    'Tranfer Total', 'Receiver Id', 'Receiver Name',
                                    'Num Deposits', 'Deposit Total'])
display(layering_df)

,Sender Id,Sender Name,Num Transfers,Tranfer Total,Receiver Id,Receiver Name,Num Deposits,Deposit Total
0,85,Karla Underhill,1,190064.95130,43,Rufus Bryant,1,150767.24250
1,72,Shannon Bloom,1,172315.88870,63,Sharon Ward,1,165348.34170
2,29,Nicholas Morris,1,145813.44520,12,Mike Locke,1,137616.85760
3,89,Leslie Potter,1,141574.78840,65,Julius Leigh,1,121319.15480
4,51,Carina Pitt,1,137260.16520,17,Julius Walsh,1,129776.17210
5,25,Carol Ebbs,1,99068.57627,18,Peter Bailey,1,88033.75021


### Monitor rapid in and out transfers
This analysis identifies anyone who received more than X in the last n days <br>
and find all the rapid outgoing transfers of high percentage of incoming transfers. 

In [12]:
def find_rapid_transfers(curDate, lookback, depositThreshold, sort="TotalDeposits", order="DESC", limit=10, skip=0):
    """
    This function looks for anyone who received more than a desired deposit threshold
    in last n days, and finds all the rapid outgoing transfers of high percentage of receipts. 

    Parameters:
        curDate : str (yyyy-mm-dd)
            Reference data for the analysis
        lookback : int
            number of lookback days.
        depositThreshold : int
            The total deposits (incoming transfers) above this threshold are examined.
        sort : str, default 'amtTransfer'
            Data field used to sort the result.
        order : str, default 'DESC'
            Define the sprting order of the results. 'DESC' or 'ASC'
        limit : int, default 10
            Maximum number of records top records returned.
        skip : int, defalut 0
            Number of records to be trimmed from the top.
    
    Returns: list of lists
    """

    def read_neo4j(tx, curDate, lookback, depositThreshold, sort, order, limit, skip):

        cypher_query = """
            // Find anyone who received transfers more than X in the last n days
            WITH date('{0}') as curDate
            WITH curDate, curDate-duration({{days:{1}}}) as curDate30
            MATCH (ah:AccHolder)-[:HAS_BANKACCOUNT]->(recAcc:BankAccount)<-[:SEND]-(trf:BankTransfer)
            WHERE trf.transferDate >= curDate30
            AND trf.transferDate <= curDate
            WITH recAcc.accNum as accountNumber,
                min(trf.transferDate) as StartDate,
                max(trf.transferDate) as EndDate,
                duration.inDays(min(trf.transferDate),max(trf.transferDate)).days as NumDays,
                count(trf.amount) as NumDeposits,
                sum(trf.amount) as TotalDeposits
            WHERE TotalDeposits > {2}
            WITH collect({{accNum: accountNumber,
                            startDate: StartDate,
                            endDate: EndDate,
                            numDays: NumDays,
                            numDeposits: NumDeposits,
                            totDeposits: TotalDeposits}}) as aggregators
                            
            // find all the rapid outgoing transfers of high percentage of incoming transfers
            UNWIND aggregators as curAgg
            MATCH (ba:BankAccount)-[:SEND]->(trf:BankTransfer)
            WHERE ba.accNum = curAgg.accNum
            AND trf.transferDate >= curAgg.startDate
            AND trf.transferDate <= curAgg.endDate+duration({{days:10}})
            WITH ba.accNum as accountNumber,
                curAgg.startDate as StartDate,
                max(trf.transferDate) as EndDate,
                duration.inDays(curAgg.startDate,max(trf.transferDate)).days as NumDays,
                curAgg.numDeposits as NumDeposits,
                curAgg.totDeposits as TotalDeposits,
                count(trf.amount) as NumWithdrawals,
                sum(trf.amount) as TotalWithdrawals,
                abs(sum(trf.amount))/curAgg.totDeposits as pctWithdrawal
            WHERE pctWithdrawal > 0.75 AND pctWithdrawal <= 1.0

            // retrieve the account holder info
            MATCH(ah:AccHolder)-[:HAS_BANKACCOUNT]->(ba:BankAccount)
            WHERE ba.accNum = accountNumber
            RETURN ah.cif AS cif, ah.name AS custName, ba.accNum AS accNum,
                    StartDate, EndDate,
                    NumDeposits, TotalDeposits,
                    NumWithdrawals, TotalWithdrawals, pctWithdrawal
            ORDER BY {3} {4}
            SKIP $skip
            LIMIT $limit   
        """.format(curDate, lookback, depositThreshold, sort, order)
        # Run the statement within the transaction
        result = tx.run(cypher_query, limit=limit, skip=skip)

        # Extract records from the Result
        return result.values('cif', 'custName', 'accNum', 'StartDate'
                            , 'EndDate', 'NumDeposits', 'TotalDeposits'
                            , 'NumWithdrawals', 'TotalWithdrawals', 'pctWithdrawal' )
    
    with driver.session() as session:
        return session.execute_read(read_neo4j, curDate, lookback, depositThreshold, sort, order, limit, skip)

The analysis below examines total deposits over 100k during the last 30 days from 2021-06-30,<br>
and looks for rapid outgoing transfers within 75% of the deposits.

In [13]:
rapid_transfers = find_rapid_transfers(curDate='2021-06-30', lookback=30, depositThreshold=100000)

In [14]:
rapid_transfers_df = pd.DataFrame(rapid_transfers, columns = ['CIF', 'Cust Name', 'Acc Num', 'Start Date'
                            , 'End Date', 'Num Deposits', 'Total Deposits'
                            , 'Num Withdrawals', 'Total Withdrawals', 'pct Withdrawal'])
display(rapid_transfers_df)

,CIF,Cust Name,Acc Num,Start Date,End Date,Num Deposits,Total Deposits,Num Withdrawals,Total Withdrawals,pct Withdrawal
0,36,Shannon Rixon,855-21-8447,2021-06-09,2021-06-25,2,262115.83235,3,258895.313050,0.987713
1,74,Winnie Murphy,857-13-2148,2021-06-01,2021-06-04,2,189147.97027,1,157691.740100,0.833695
2,26,Roger Mcnally,025-44-7058,2021-06-23,2021-06-27,1,145813.44520,1,137616.857600,0.943787
3,8,Candace Shea,820-62-2326,2021-06-07,2021-06-26,2,121489.34234,2,117016.590903,0.963184


### Analyse Foreign Remittances
For this analysis, I considered the customer's country as the location of their bank. <br>
In this case, any transfer to a receiver from a different country is considered a foreign remittance.<br>
The analysis below identifies the countries between which the foreign remittances are high.


In [15]:
def examine_foreign_remittances(sort='totalTransfer', order='DESC', limit=10, skip=0):
    """
    This function calculates the total transfers between two countries (countries of the customer),
    and returns the top n country pairs with highest total transfers.

    Parameters:
        sort : str, default 'amtTransfer'
            Data field used to sort the result.
        order : str, default 'DESC'
            Define the sprting order of the results. 'DESC' or 'ASC'
        limit : int, default 10
            Maximum number of records top records returned.
        skip : int, defalut 0
            Number of records to be trimmed from the top.

    Returns: list of lists

    """
    def read_neo4j(tx, sort, order, limit, skip):
        # Define the cypher statement
        cypher_query = """
            MATCH (c1:Country)<-[:CITIZEN_OF]-(ah1:AccHolder)-[:HAS_BANKACCOUNT]->(sender:BankAccount)-[:SEND]->
            (trf:BankTransfer)-[:SEND]->(receiver:BankAccount)<-[:HAS_BANKACCOUNT]-(ah2:AccHolder)-[:CITIZEN_OF]->(c2:Country)
            // remove local transfers
            WHERE c1.name <> c2.name
            WITH c1.name AS senderCountry, c2.name as ReceiverCountry
                ,count(trf.amount) as numTransfers, sum(trf.amount) as totalTransfer
            RETURN senderCountry, ReceiverCountry, numTransfers, totalTransfer
            ORDER BY {0} {1}
            SKIP $skip
            LIMIT $limit
        """.format(sort, order)

        # Run the statement within the transaction
        result = tx.run(cypher_query, limit=limit, skip=skip)

        # Extract records from the Result
        return result.values('senderCountry', 'ReceiverCountry', 'numTransfers', 'totalTransfer')
    
    with driver.session() as session:
        return session.execute_read(read_neo4j, sort, order, limit, skip)

In [16]:
foreign_remittances = examine_foreign_remittances()

In [17]:
foreign_remittances_df = pd.DataFrame(foreign_remittances, columns = ['Sender Country', 'Receiver Country', 'Num Transfers', 'Total Transfers'])
display(foreign_remittances_df)

,Sender Country,Receiver Country,Num Transfers,Total Transfers
0,Mauritania,Comoros,2,388346.97780
1,Malawi,Grenada,3,377344.58945
2,Iraq,Saint Kitts And Nevis,2,367355.26180
3,Japan,Macedonia,2,338851.77070
4,Suriname,Guinea-bissau,3,331792.25345
5,Luxembourg,Mauritius,3,321097.15182
6,Pakistan,Macedonia,2,318492.54300
7,Luxembourg,Maldives,2,314528.69970
8,Japan,Italy,2,311968.44810
9,Pakistan,Grenada,2,311637.94030
